In [1]:
from plyfile import PlyData, PlyElement
import numpy as np
import random
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import tqdm
import laspy

In [2]:
def read_label_plymesh(filename):
    """Read a .ply file and extract vertex and face data with attributes."""
    plydata = PlyData.read(filename)
    
    # Extract vertex data
    vertex_data = plydata['vertex'].data
    data = {
        'x': vertex_data['x'],
        'y': vertex_data['y'],
        'z': vertex_data['z'],
        'red': vertex_data['red'],
        'green': vertex_data['green'],
        'blue': vertex_data['blue'],
        'label': vertex_data['label'] if 'label' in vertex_data.dtype.names else None,
        'instance_id': vertex_data['instance_id'] if 'instance_id' in vertex_data.dtype.names else None,
    }
    vertices = np.array([data['x'], data['y'], data['z'], data['red'], data['green'], data['blue'], 
                         data['label'], data['instance_id']]).T
    
    # Extract face data if available
    faces = None
    if 'face' in plydata:
        faces = np.vstack(plydata['face'].data['vertex_indices'])  # Ensure it's in the correct shape (N, 3)
    
    return vertices, faces

def assign_random_colors(vertices):
    # Check if 'instance_id' exists (assumed in the 8th column if present)
    instance_id_index = 7 if vertices.shape[1] == 8 else None
    if instance_id_index is None:
        raise ValueError("Vertices data does not contain 'instance_id' information.")
    
    # Extract instance IDs
    instance_ids = vertices[:, instance_id_index]
    unique_ids = np.unique(instance_ids)
    
    # Create a color map with random colors for each unique instance ID
    color_map = {instance_id: (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)) for instance_id in unique_ids}
    
    # Assign random colors to each vertex based on instance ID
    colors = np.array([color_map[instance_id] for instance_id in instance_ids])
    vertices[:, 3:6] = colors  # Replace RGB columns with new random colors

    return vertices


def save_colored_ply(vertices, faces, filename, with_label=True):
    """Save an RGB point cloud as a PLY file with random colors for each instance_id."""
    assert vertices.ndim == 2

    if with_label:
        if vertices.shape[1] == 7:
            python_types = (float, float, float, int, int, int, int)
            npy_types = [('x', 'f4'), ('y', 'f4'), ('z', 'f4'), ('red', 'u1'), ('green', 'u1'),
                         ('blue', 'u1'), ('label', 'u4')]
        elif vertices.shape[1] == 8:
            python_types = (float, float, float, int, int, int, int, int)
            npy_types = [('x', 'f4'), ('y', 'f4'), ('z', 'f4'), ('red', 'u1'), ('green', 'u1'),
                         ('blue', 'u1'), ('label', 'u4'), ('instance_id', 'u4')]

    vertices_list = []
    for row_idx in range(vertices.shape[0]):
        cur_point = vertices[row_idx]
        vertices_list.append(tuple(dtype(point) for dtype, point in zip(python_types, cur_point)))
    vertices_array = np.array(vertices_list, dtype=npy_types)
    elements = [PlyElement.describe(vertices_array, 'vertex')]

    # Add face elements if available
    if faces is not None:
        faces_array = np.empty(len(faces), dtype=[('vertex_indices', 'i4', (3,))])
        faces_array['vertex_indices'] = faces
        elements.append(PlyElement.describe(faces_array, 'face'))

    # Write to file
    PlyData(elements).write(filename)

In [10]:
vertices, faces = read_label_plymesh('../../data/scannet200/val/scene0700_00.ply')
instance_ids = vertices[:, 7]
print('Number of unique instance IDs:', len(np.unique(instance_ids)))

Number of unique instance IDs: 36


In [9]:
semantics = np.load('../../data/scannet/ssfm_valid/scene0700_00/associations/semantic_points.npy')
print('Number of unique semantic labels:', len(np.unique(semantics)))

Number of unique semantic labels: 37


In [14]:
def convert_to_las(vertices, las_filename):
    # Check if 'instance_id' exists (assumed in the 8th column if present)
    instance_id_index = 7 if vertices.shape[1] == 8 else None
    if instance_id_index is None:
        raise ValueError("Vertices data does not contain 'instance_id' information.")

    # Create a .las file and write the combined points to it
    header = laspy.LasHeader(point_format=3, version="1.2")
    header.scale = [0.001, 0.001, 0.001]

    las = laspy.LasData(header)
    las.x = vertices[:, 0]
    las.y = vertices[:, 1]
    las.z = vertices[:, 2]

    # assign instance id to las intensity
    las.intensity = vertices[:, instance_id_index]

    las.write(las_filename)
    

In [17]:
label_folder = '../../data/scannet200/val'
save_folder = '../../data/scannet/ssfm_valid'
scan_folder = '../../data/scannet/scans'
label_ply_files = [os.path.join(label_folder, f) for f in os.listdir(label_folder) if f.endswith('.ply')]

for label_ply_file in tqdm.tqdm(label_ply_files):
    vertices, faces = read_label_plymesh(label_ply_file)
    scene_name = os.path.basename(label_ply_file).split('.')[0]
    scan_scene_folder = os.path.join(scan_folder, scene_name)
    save_scene_folder = os.path.join(save_folder, scene_name, 'reconstructions')
    if not os.path.exists(save_scene_folder):
        continue

    original_ply = os.path.join(scan_scene_folder, scene_name + '_vh_clean_2.ply')
    plydata = PlyData.read(original_ply)
    vertex_data = plydata['vertex'].data

    # replace the xyz
    vertices[:, 0] = vertex_data['x']
    vertices[:, 1] = vertex_data['y']
    vertices[:, 2] = vertex_data['z']

    # Assign random colors based on instance ID
    colored_vertices = assign_random_colors(vertices)
    convert_to_las(colored_vertices, os.path.join(save_scene_folder, scene_name + '.las'))

    output_file = os.path.join(save_scene_folder, scene_name + '.ply')

    # Save the new PLY file with random colors and reuse faces
    save_colored_ply(colored_vertices, faces, output_file)


100%|██████████| 312/312 [50:43<00:00,  9.76s/it] 
